In [1]:
import os
import numpy as np
from nn.Activations import *
import tensorflow as tf

In [63]:
class NN(): 
    
    def __init__(self, hidden, in_dim, out_dim, activation, loss_fn):
        
        self.loss_fn = loss_fn
        self.num_layers = len(hidden)+1
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.activation = activation
        self.weights = []
        self.biases = []
        self.layers = []
        self.layer_grads = []
        
        for i in range(self.num_layers):
            if i==0:
                self.weights.append(tf.random.normal(shape=(in_dim,hidden[i])))
                self.biases.append(tf.random.normal(shape=(hidden[i],)))
            elif i==self.num_layers-1:
                self.weights.append(tf.random.normal(shape=(hidden[i-1],out_dim)))
                self.biases.append(tf.random.normal(shape=(out_dim,)))
            else:
                self.weights.append(tf.random.normal(shape=(hidden[i-1],hidden[i])))
                self.biases.append(tf.random.normal(shape=(hidden[i],)))
    
    def forward(self, x):
        
        for i in range(self.num_layers):
            if i==0:
                z = tf.add(tf.matmul(x,self.weights[i]),self.biases[i])
            else:
                z = tf.add(tf.matmul(self.layers[i-1],self.weights[i]),self.biases[i])
            a = activate(z, function=self.activation)
            self.layers.append(a)
            self.layer_grads.append(a)
        return self.layers[-1]
    
    def backward(self, x, y, lr=0.001):
        self.errors = []
        for i in range(self.num_layers-1,-1,-1):
            if i==self.num_layers-1:
                if self.loss_fn == 'mse':
                    err = tf.square(y-self.layers[i])
                self.errors.append(tf.reduce_mean(err))
                err_delta = err*activate(err, self.activation, grad=True)
            else:
                err = tf.tensordot(self.layer_grads[i+1],tf.transpose(self.weights[i+1]),axes=0)
                err_delta = activate(err, self.activation, grad=True)
                self.errors.append(err_delta)
                
            self.layer_grads[i] = err_delta
        
        for i in range(self.num_layers-1,0,-1):
            self.weights[i] -= lr * tf.tensordot(tf.transpose(self.layers[i-1]),(self.layer_grads[i]),axes=0)
            self.biases[i] -= lr * self.layer_grads[i]
        print(self.layer_grads)
        self.weights[0] -= lr * tf.tensordot(tf.transpose(x[np.newaxis]),(self.layer_grads[0]),axes=0)
        self.biases[0] -= lr * self.layer_grads[0]

In [64]:
model = NN([4,8],3,1,'relu','mse')

In [65]:
sess = tf.Session()

In [66]:
sess.run(tf.global_variables_initializer())

In [67]:
num_instances = 100
inputs =  np.float32(np.random.randint(low=0, high=10, size=(num_instances,3)))
outputs = inputs * 10 + 3

In [68]:
x_train, x_test = inputs[:70], inputs[70:]
y_train, y_test = outputs[:70], outputs[70:]

In [69]:
epochs = 100
batch_size=8
for e in range(epochs):
    for b in range(0,x_train.shape[0],batch_size):
        outs = model.forward(x_train[b:b+batch_size])
        model.backward(x_train[b:b+batch_size],y_train[b:b+batch_size])
        err = model.errors[b//batch_size]
        print("Training Epoch {0:3d} Batch {1:3d} Error {3.2f}".format(e+1,(b//batch_size)+1,err))

[<tf.Tensor 'mul_56:0' shape=(8, 3, 1, 8, 8, 4) dtype=float32>, <tf.Tensor 'mul_55:0' shape=(8, 3, 1, 8) dtype=float32>, <tf.Tensor 'mul_54:0' shape=(8, 3) dtype=float32>]


ValueError: Dimensions must be equal, but are 3 and 8 for 'sub_21' (op: 'Sub') with input shapes: [3,4], [3,8,1,8,3,1,8,8,4].

In [10]:
model.layers, model.num_layers

([<tf.Tensor 'mul:0' shape=(100, 4) dtype=float32>,
  <tf.Tensor 'mul_1:0' shape=(100, 8) dtype=float32>,
  <tf.Tensor 'mul_2:0' shape=(100, 1) dtype=float32>],
 3)

In [8]:
print(sess.run(outs))

[[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]]
